In [1]:
import socket
import hmac
import numpy as np
import time

class UDPReceiver:
    def __init__(self, X, Y, key, ip='0.0.0.0', port=23422):
        self.X = X
        self.Y = Y
        self.key = key
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.bind((ip, port))
        self.buffer = [b''] * X.shape[0]
        self.macs = [b''] * X.shape[1]
        self.res = {}
        self.total_mac_length = 0
        self.total_data_length = 0
        self.total_verified_data_length = 0
        self.expected_chunk_index = 1

    def data_parser(self, data):
        chunk_index = int.from_bytes(data[:4], 'big')
        if np.sum(self.Y[chunk_index % self.X.shape[0] - 1]):
            chunk_data = data[4:-48]
            mac = data[-48:]
        else:
            chunk_data = data[4:]
            mac = b''
        return chunk_index, chunk_data, mac

    def fill_buffer_mac(self, chunk_index, chunk_data, mac):
        self.buffer[chunk_index % self.X.shape[0] - 1] = chunk_data
        try:
            self.macs[np.where(self.Y[chunk_index % self.X.shape[0] - 1] == 1)[0][0]] = mac
        except IndexError:
            pass

    def reset_buffer_mac(self):
        self.buffer = [b''] * self.X.shape[0]
        self.macs = [b''] * self.X.shape[1]

    def check_mac_for_buffer(self, excepted_chunk_index):
        buffer_array = np.array(self.buffer)  # Convert buffer to a numpy array
        buffer_verified = [b'*' * 2] * self.X.shape[0]

        for tags in range(self.X.shape[1]):
            data = b''.join(buffer_array[np.where(self.X[:, tags] == 1)[0]])  # Use numpy array for indexing
            mac = hmac.new(self.key, data, 'sha384').digest()
            if mac == self.macs[tags]:
                for x in np.where(self.X[:, tags] == 1)[0]:
                    buffer_verified[x] = self.buffer[x]

        cnt = (excepted_chunk_index - 1) - (excepted_chunk_index - 1) % self.X.shape[0]
        for x in buffer_verified:
            self.res[cnt] = x
            cnt += 1

        self.reset_buffer_mac()

    def receive_data(self):
        print("****Receiving in 2D mode******\n")
        start = time.time()

        while True:
            data, _ = self.sock.recvfrom(4096)  # 8 bytes for index + 1024 bytes of data
            if data == b'END':
                self.finalize()
                break

            chunk_index, chunk_data, mac = self.data_parser(data)
            self.total_mac_length += len(mac)

            if self.expected_chunk_index == chunk_index:
                self.fill_buffer_mac(chunk_index, chunk_data, mac)
                self.expected_chunk_index += 1
            else:
                left_to_verify = self.X.shape[0] - self.expected_chunk_index % self.X.shape[0]
                if self.expected_chunk_index % self.X.shape[0] == 0 or (
                        chunk_index - self.expected_chunk_index) > left_to_verify:
                    self.check_mac_for_buffer(self.expected_chunk_index)

                self.fill_buffer_mac(chunk_index, chunk_data, mac)
                self.expected_chunk_index = chunk_index + 1

            if chunk_index % self.X.shape[0] == 0:
                self.check_mac_for_buffer(self.expected_chunk_index)

        print("Time: ", time.time() - start)

    def finalize(self):
        print("End of transmission\n")

        received_data = b''.join([self.res[x] for x in self.res])
        print("Received data: \n", received_data.decode())

        print(f"\nStats --------------\n\nTotal tag Length: {self.total_mac_length * 8}")
        for x in self.res:
            self.total_data_length += len(self.res[x])
            if b'*' not in self.res[x]:
                self.total_verified_data_length += len(self.res[x])
        print(f"Total Data Length: {self.total_data_length * 8}")
        print(f"Total Verified Data Length: {self.total_verified_data_length * 8}")
        print(f"Goodput: {np.round(self.total_verified_data_length / (self.total_data_length + self.total_mac_length), 2)}")
        self.sock.close()


# Example usage:
X = np.array([
    [1, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
    [1, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
    [1, 0, 0, 0, 0, 0, 0, 0, 1],  # m3
    [0, 1, 0, 0, 0, 0, 1, 0, 0],  # m4
    [0, 1, 0, 0, 0, 0, 0, 1, 0],  # m5
    [0, 1, 0, 0, 0, 0, 0, 0, 1],  # m6
    [0, 0, 1, 0, 0, 0, 1, 0, 0],  # m7
    [0, 0, 1, 0, 0, 0, 0, 1, 0],  # m8
    [0, 0, 1, 0, 0, 0, 0, 0, 1],  # m9
    [0, 0, 0, 1, 0, 0, 1, 0, 0],  # m10
    [0, 0, 0, 1, 0, 0, 0, 1, 0],  # m11
    [0, 0, 0, 1, 0, 0, 0, 0, 1],  # m12
    [0, 0, 0, 0, 1, 0, 1, 0, 0],  # m13
    [0, 0, 0, 0, 1, 0, 0, 1, 0],  # m14
    [0, 0, 0, 0, 1, 0, 0, 0, 1],  # m15
    [0, 0, 0, 0, 0, 1, 1, 0, 0],  # m16
    [0, 0, 0, 0, 0, 1, 0, 1, 0],  # m17
    [0, 0, 0, 0, 0, 1, 0, 0, 1]   # m18
])

Y = np.array([
    [0, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
    [0, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
    [1, 0, 0, 0, 0, 0, 0, 0, 0],  # m3
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m4
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m5
    [0, 1, 0, 0, 0, 0, 0, 0, 0],  # m6
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m7
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m8
    [0, 0, 1, 0, 0, 0, 0, 0, 0],  # m9
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m10
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m11
    [0, 0, 0, 1, 0, 0, 0, 0, 0],  # m12
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m13
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m14
    [0, 0, 0, 0, 1, 0, 0, 0, 0],  # m15
    [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m16
    [0, 0, 0, 0, 0, 1, 0, 0, 0],  # m17
    [0, 0, 0, 0, 0, 0, 0, 0, 1]   # m18
])

receiver = UDPReceiver(X, Y, key=b'key')
receiver.receive_data()


****Receiving in 2D mode******



In [1]:

import cryptography.hazmat.primitives.padding as padding
import numpy as np
import socket
import hmac
import pprint


class UDP_TX:
    def __init__(self, IP:str ='0.0.0.0', PORT:int = 23422, X = np.eye(10), Y = np.eye(10), chunk_size_Byte=128, KEY=b"key", digestmod='sha384'):
        self.IP = IP
        self.PORT = PORT
        self.X = X
        self.Y = Y
        self.chunk_size_Byte = chunk_size_Byte
        self.KEY = KEY
        self.digestmod = digestmod

    # Padding using PKCS7
    def pad(self,data, chunk_size_Byte = None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        
        padder = padding.PKCS7(chunk_size_Byte*8).padder()
        padded_data = padder.update(data) + padder.finalize()
        return padded_data


    def divide_Bytes_and_pad(self,Bytes, chunk_size_Byte=None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        
        chunks = [Bytes[i:i + chunk_size_Byte] for i in range(0, len(Bytes), chunk_size_Byte)]
        if len(chunks[-1]) != chunk_size_Byte:
            chunks[-1] = self.pad(chunks[-1], chunk_size_Byte)
        return chunks

    def chucks_to_nD_arrangments(self,chunks,chunk_size_Byte = None, X= None):
        if chunk_size_Byte is None:
            chunk_size_Byte = self.chunk_size_Byte
        if X is None:
            X = self.X

        nD_arrangments = [chunks[i:i + X.shape[0]] for i in range(0, len(chunks), X.shape[0])]
        if len(nD_arrangments[-1]) != X.shape[0]:
            for i in range( X.shape[0] - len(nD_arrangments[-1]))  :
                nD_arrangments[-1].append(self.pad(b'',chunk_size_Byte=chunk_size_Byte))
        return np.array(nD_arrangments)

    def mac_for_page(self,page, key = None, X = None, Y = None):
        if key is None:
            key = self.KEY
        if X is None:
            X = self.X
        if Y is None:
            Y = self.Y
        
        res = {}
        for msg_index in range(X.shape[0]):
            res[msg_index] = page[msg_index]
        
        for tag_index in range(X.shape[1]):
            selected_blocks = page[X[:, tag_index] == 1]
            if selected_blocks.size > 0:
                data = b''.join(selected_blocks)
                target_index = np.where(Y[:, tag_index] == 1)[0][0]
                res[target_index] = page[target_index] + hmac.new(key, data, digestmod=self.digestmod).digest()
        return res

    def send_msg(self,SN, msg, sock, dest):
        sock.sendto(SN.to_bytes(4, 'big') + msg, dest)


    def transmit(self, Bytes, attack:list=[]):

        SN = 0
        cnt = 0
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            chunks = self.divide_Bytes_and_pad(Bytes = Bytes, chunk_size_Byte= self.chunk_size_Byte)
            pages = self.chucks_to_nD_arrangments(chunks = chunks, chunk_size_Byte= self.chunk_size_Byte, X =  self.X)

            for i in range(len(pages)):
                page = self.mac_for_page(page=pages[i], key= self.KEY, X= self.X, Y= self.Y)
                for msg in page.values():
                    if SN in attack:
                        SN += 1
                        continue
                    self.send_msg(SN, msg, sock, (self.IP,  self.PORT))
                    cnt += 1
                    SN += 1
            ### send the end message this is not secure but it is just for testing
            sock.sendto(b'END', ( self.IP,  self.PORT))
        return cnt

# unit test




IP = "0.0.0.0"
PORT = 23422

X = np.eye(3)
Y = np.eye(3)

chunk_size_Byte = 7

key = b"key"

upd_tx = UDP_TX(IP, PORT, X, Y, chunk_size_Byte, key)

data = b'This test shows 2D integrity check is better than blockwise integrity.'

# testing the divide_Bytes_and_pad function

print("Dividing the Byte data every to chunk_size_Bytes and pad the last chunck if needed",upd_tx.divide_Bytes_and_pad(data, chunk_size_Byte=chunk_size_Byte))

# testing the chucks_to_nD_arrangments function witht X.shape[0] and chunk_size
chunks = upd_tx.divide_Bytes_and_pad(data, chunk_size_Byte=chunk_size_Byte)
print(upd_tx.chucks_to_nD_arrangments(chunks, chunk_size_Byte, X))

# testing the mac_for_page function
pages = upd_tx.chucks_to_nD_arrangments(chunks, chunk_size_Byte, X)
pprint.pprint(upd_tx.mac_for_page(pages[0], key, X, Y))

# at this point after calculating the MAC for each page, we can send the data
# iterate through the chunks (msg) in every page and send it to the destination
# the function send_msg is responsible for sending the data to the destination
# the function transmit input bytes and create pages with padded chunks  and 
# then iterates through the all the chunks in the pages and send them to the destination

# testing the transmit function
print("Number of messages sent: ", upd_tx.transmit(data))
# testing the 
print("Number of messages sent under attack: ", upd_tx.transmit(data, attack=[1,3]))






Dividing the Byte data every to chunk_size_Bytes and pad the last chunck if needed [b'This te', b'st show', b's 2D in', b'tegrity', b' check ', b'is bett', b'er than', b' blockw', b'ise int', b'egrity.']
[[b'This te' b'st show' b's 2D in']
 [b'tegrity' b' check ' b'is bett']
 [b'er than' b' blockw' b'ise int']
 [b'egrity.' b'\x07\x07\x07\x07\x07\x07\x07'
  b'\x07\x07\x07\x07\x07\x07\x07']]
{0: b'This te\xd2\x1b\x97n\xe9\xf9\x10\xc8f\xd6\xac-e\x91\xfc\xa8\xddpt\xb6\x1b'
    b'>\x98f%\x88? F\\,j\xde-v3"]\xcf|\x1a\xcf\xcbu\x19\xcfc3',
 1: b'st show\n\xad46@\x9c\xce\x82\x1a\xa0\x17\xb8\xed\xd8N\x1d\xd2'
    b'\x92\xc8Z\xb4\xae\x90\xa4*\xb6\x1f\xbb\xedAF\n\x11\xd4@\x9e\xdbeDC\x80'
    b'\xf3\x13\x00R\xdbM\xfd',
 2: b's 2D in\x17\\\xc3\x00:2\x08y\x1ce\xf2\xc9\xb5\xb4/\x96\x14\xc5\xee\x8eS'
    b'\xd4\xb7\x8c\xf9B\xff\xb2\xc2s\xa97\xa7q\xfc\xefU<\x82\xf9\xf8mC)\x0e'
    b'\xc7\x0cf'}
Number of messages sent:  12
Number of messages sent under attack:  10
